In [2]:
import pyspark
from pyspark.sql import SparkSession
from os import walk
from Helper_Code.helpfull_functions import *

In [3]:
spark = SparkSession\
    .builder\
    .appName("APPNAME")\
    .enableHiveSupport()\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/17 15:21:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/17 15:21:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [44]:
care_site = merge_files("/home/jupyter/omop-ed-datapipeline/care_site")

(2689, 6)
+------------+--------------+---------------------------+--------------------+----------------------+-----------------------------+
|care_site_id|care_site_name|place_of_service_concept_id|         location_id|care_site_source_value|place_of_service_source_value|
+------------+--------------+---------------------------+--------------------+----------------------+-----------------------------+
|          44|           YNH|                          0|ca62e9a2a98e68419...|      EMH HOSPITAL LAB|                          Lab|
|         192|            YM|                          0|ca62e9a2a98e68419...|  EMC CHILD PSYCHIATRY|         Child & Adolescen...|
|         352|            GH|                       null|6fb5c426017a739f8...|   CHILD LIFE SERVICES|                         null|
|         355|            GH|                       null|6fb5c426017a739f8...|       CASE MANAGEMENT|                         null|
|         362|            BH|                       null|82a71069f

In [13]:
concept = merge_files("/home/jupyter/omop-ed-datapipeline/concept", spark)

(8107890, 10)
+----------+--------------------+---------+----------------+-----------------+----------------+------------+----------------+--------------+--------------+
|concept_id|        concept_name|domain_id|   vocabulary_id| concept_class_id|standard_concept|concept_code|valid_start_date|valid_end_date|invalid_reason|
+----------+--------------------+---------+----------------+-----------------+----------------+------------+----------------+--------------+--------------+
|  40744178|Magnesium glycero...|     Drug|RxNorm Extension|     Branded Drug|               S| OMOP4712893|      2018-06-04|    2099-12-31|          null|
|  40744177|Magnesium glycero...|     Drug|RxNorm Extension| Marketed Product|               S| OMOP4712894|      2018-06-04|    2099-12-31|          null|
|  40744176|Magnesium glycero...|     Drug|RxNorm Extension| Branded Drug Box|               S| OMOP4712895|      2018-06-04|    2099-12-31|          null|
|  40744175|Magnesium glycero...|     Drug|RxNorm 

In [16]:
concept2= concept.where(concept.concept_name.contains('triage'))
concept2.show()

+----------+--------------------+------------+-------------+--------------------+----------------+---------------+----------------+--------------+--------------+
|concept_id|        concept_name|   domain_id|vocabulary_id|    concept_class_id|standard_concept|   concept_code|valid_start_date|valid_end_date|invalid_reason|
+----------+--------------------+------------+-------------+--------------------+----------------+---------------+----------------+--------------+--------------+
|  44805767|Referral to ear, ...| Observation|       SNOMED|           Procedure|               S|818841000000106|      2012-04-01|    2099-12-31|          null|
|   3559177|Referral to ear, ...| Observation|       SNOMED|           Procedure|            null|818851000000109|      1970-01-01|    2012-04-01|             U|
|  44809126|Referral to ortho...| Observation|       SNOMED|           Procedure|               S|835201000000105|      2012-10-01|    2099-12-31|          null|
|  44809425|Referral to spin

In [6]:
concept_ancestor = merge_files("/home/jupyter/omop-ed-datapipeline/concept_ancestor")

(78498725, 4)
+-------------------+---------------------+------------------------+------------------------+
|ancestor_concept_id|descendant_concept_id|min_levels_of_separation|max_levels_of_separation|
+-------------------+---------------------+------------------------+------------------------+
|           45782152|             21162043|                       2|                       3|
|           45787425|             43772033|                       2|                       3|
|           45799162|             43036684|                       2|                       3|
|           45784838|             44049892|                       2|                       2|
|           45780084|             41134200|                       2|                       3|
|           45802631|              2055112|                       2|                       2|
|           45797896|             35886435|                       2|                       2|
|           45793293|             42949676|   

In [7]:
concept_class = merge_files("/home/jupyter/omop-ed-datapipeline/concept_class")

(415, 3)
+-------------------+--------------------+------------------------+
|   concept_class_id|  concept_class_name|concept_class_concept_id|
+-------------------+--------------------+------------------------+
|    Qualifier Value|     Qualifier Value|                44819021|
|     Quality Metric|      Quality Metric|                44819092|
|               Race|                Race|                44819030|
|               Read|                Read|                44819273|
|    Record Artifact|     Record Artifact|                44818999|
|       Relationship|   OMOP Relationship|                44819274|
|Respiratory Disease|Diseases of the r...|                44819176|
|       Revenue Code|        Revenue Code|                44819075|
|       Skin Disease|Diseases of the s...|                44819178|
|                SMQ|Standardized Medd...|                44819060|
|                SOC|  System Organ Class|                44819037|
|     Social Context|      Social Conte

In [8]:
concept_relationship = merge_files("/home/jupyter/omop-ed-datapipeline/concept_relationship")

(53237604, 6)
+------------+------------+---------------+----------------+--------------+--------------+
|concept_id_1|concept_id_2|relationship_id|valid_start_date|valid_end_date|invalid_reason|
+------------+------------+---------------+----------------+--------------+--------------+
|    41508690|    40808454| Has brand name|      2017-07-18|    2099-12-31|          null|
|    19025908|    19023073| Has brand name|      2016-10-07|    2099-12-31|          null|
|    43645720|    43574063| Has brand name|      2016-07-01|    2099-12-31|          null|
|    43605772|    43572501| Has brand name|      2016-07-01|    2099-12-31|          null|
|    43269292|    43254087| Has brand name|      2016-09-30|    2099-12-31|          null|
|    40846778|    43570581| Has brand name|      2017-07-18|    2099-12-31|          null|
|    44093687|    44017800| Has brand name|      2017-06-25|    2099-12-31|          null|
|    45892878|    19068457| Has brand name|      2016-10-07|    2099-12-31| 

In [9]:
concept_synonym = merge_files("/home/jupyter/omop-ed-datapipeline/concept_synonym")

(2677157, 3)
+----------+-------------------------------------+-------------------+
|concept_id|                 concept_synonym_name|language_concept_id|
+----------+-------------------------------------+-------------------+
|  42205956|  투시하 이물제거술(상부소화관이물...|            4175771|
|  42205959|   기관또는기관지봉합술(경부접근),...|            4175771|
|  42205964|   절개술(안면과경부이외,제2범위부...|            4175771|
|  42205966|마스크에의한폐쇄순환식전신마취유지...|            4175771|
|  42205969|   동맥류절제술[혈관이식술포함]-하...|            4175771|
|  42205970|   사지골절정복술[복잡골절포함]-관...|            4175771|
|  42205978|   신우,요관성형술, 권역외상센터중...|            4175771|
|  42205986|       폐쐐기절제술(2~3개), 권역외...|            4175771|
|  42205988|   간절제술-간엽절제, 권역외상센터...|            4175771|
|  42205991|  쇄골골절고정술, 권역외상센터중증...|            4175771|
|  42205993|    혈관결찰술(기타), 권역외상센터...|            4175771|
|  42205997|    인두이물제거술(단순[편도상와])...|            4175771|
|  42205999|   전자궁적출술(림프절절제를 하지 ...|            4175771|
|  42206000|   피판작성술-피부-원거리-수부이외

In [10]:
condition_occurrence = merge_files("/home/jupyter/omop-ed-datapipeline/condition_occurrence")

(59801967, 17)


22/06/07 18:13:11 WARN DAGScheduler: Broadcasting large task binary with size 1710.9 KiB
22/06/07 18:13:11 WARN DAGScheduler: Broadcasting large task binary with size 1710.9 KiB


+---------+-----------------------+------------------------------+--------------------+--------------------+------------------------+------------------+----------------------+-------------------------+---------------------------+-----------+-----------+-------------------+---------------+----------------------+---------------------------+-----------------------------+
|person_id|condition_occurrence_id|condition_occurrence_source_id|condition_concept_id|condition_start_date|condition_start_datetime|condition_end_date|condition_end_datetime|condition_type_concept_id|condition_status_concept_id|stop_reason|provider_id|visit_occurrence_id|visit_detail_id|condition_source_value|condition_source_concept_id|condition_status_source_value|
+---------+-----------------------+------------------------------+--------------------+--------------------+------------------------+------------------+----------------------+-------------------------+---------------------------+-----------+-----------+-----

In [11]:
csn_map = merge_files("/home/jupyter/omop-ed-datapipeline/csn_map")

(5163252, 2)
+-------------------+--------+
|visit_occurrence_id|     csn|
+-------------------+--------+
|            2300962|80127093|
|            5545132|83800838|
|            5587027|84707888|
|            5637133|84707562|
|            5811464|84461443|
|            5843367|84401342|
|            5875153|84424156|
|            5903271|84427474|
|            5933440|84433633|
|            5935435|84436258|
|            6070814|83913972|
|            6078174|84991842|
|            6083802|84632597|
|            6086650|84632655|
|            6094816|84956336|
|            6139406|84507917|
|            6140286|84631425|
|            6140288|84631532|
|            6147863|84922709|
|            6150423|84632605|
+-------------------+--------+
only showing top 20 rows



In [12]:
domain = merge_files("/home/jupyter/omop-ed-datapipeline/domain")

(48, 3)
+-------------------+--------------------+-----------------+
|          domain_id|         domain_name|domain_concept_id|
+-------------------+--------------------+-----------------+
|               Cost|                Cost|           581456|
|   Condition/Device|    Condition/Device|              235|
|             Gender|              Gender|                2|
|               Race|                Race|                3|
|          Ethnicity|           Ethnicity|                4|
|           Metadata|            Metadata|                7|
|              Visit|               Visit|                8|
|          Procedure|           Procedure|               10|
|           Modifier|            Modifier|               12|
|               Drug|                Drug|               13|
|              Route|Route Of Administ...|               15|
|               Unit|                Unit|               16|
|             Device|              Device|               17|
|          Condi

In [13]:
drug_exposure = merge_files("/home/jupyter/omop-ed-datapipeline/drug_exposure")

(17548618, 27)


22/06/07 18:13:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/06/07 18:13:30 WARN DAGScheduler: Broadcasting large task binary with size 1201.7 KiB
22/06/07 18:13:31 WARN DAGScheduler: Broadcasting large task binary with size 1201.7 KiB


+---------+----------------+---------------+------------------------+----------------------------+----------------------+--------------------------+-----------------+--------------------+-----------+-------+--------+-----------+--------------------+----------------+----------+-----------+-------------------+---------------+----------------------+----------------------+------------------------------+------------------------------+----------------------+------------------+----------------------+-----------------------+
|person_id|drug_exposure_id|drug_concept_id|drug_exposure_start_date|drug_exposure_start_datetime|drug_exposure_end_date|drug_exposure_end_datetime|verbatim_end_date|drug_type_concept_id|stop_reason|refills|quantity|days_supply|                 sig|route_concept_id|lot_number|provider_id|visit_occurrence_id|visit_detail_id|drug_source_value_code|drug_source_value_name|drug_generic_source_value_code|drug_generic_source_value_name|drug_source_concept_id|route_source_value|do

In [14]:
drug_strength = merge_files("/home/jupyter/omop-ed-datapipeline/drug_strength")

(2918520, 12)
+---------------+---------------------+------------+----------------------+---------------+-------------------------+-----------------+---------------------------+--------+----------------+--------------+--------------+
|drug_concept_id|ingredient_concept_id|amount_value|amount_unit_concept_id|numerator_value|numerator_unit_concept_id|denominator_value|denominator_unit_concept_id|box_size|valid_start_date|valid_end_date|invalid_reason|
+---------------+---------------------+------------+----------------------+---------------+-------------------------+-----------------+---------------------------+--------+----------------+--------------+--------------+
|       36268263|              1119510|        null|                  null|            1.5|                     8576|             null|                       8587|    null|      2018-08-02|    2099-12-31|          null|
|       43189893|              1351557|         8.0|                  8576|           null|               

In [15]:
location = merge_files("/home/jupyter/omop-ed-datapipeline/location")

(396108, 11)
+--------------------+--------------------+---------+--------------+-----+-----+------------+--------------------+---------------------+--------+---------+
|         location_id|           address_1|address_2|          city|state|  zip|      county|             country|location_source_value|latitude|longitude|
+--------------------+--------------------+---------+--------------+-----+-----+------------+--------------------+---------------------+--------+---------+
|00034230c7f4692ed...| 49 WILSON ST 2ND FL|     null|     NEW HAVEN|   CT|06519|   NEW HAVEN|United States of ...|                 null|    null|     null|
|000543580bd72c55d...|511 Woodward Ave ...|     null|     NEW HAVEN|   CT|06512|   NEW HAVEN|United States of ...|                 null|    null|     null|
|0005da45a893db89e...|35 HAMDEN HILLS D...|     null|        HAMDEN|   CT|06518|   NEW HAVEN|United States of ...|                 null|    null|     null|
|00170478b87d69dc3...|1173 Quinnipac av...|     nul

In [4]:
measurement = merge_files("/home/jupyter/omop-ed-datapipeline/measurement", spark)

(114649770, 24)


22/06/17 15:26:48 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB
22/06/17 15:26:49 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB


+---------+---------------+-----------+----------------------+----------------+--------------------+-------------------+----------------+---------------------------+-------------------+---------------+---------------+-------------------+---------------+---------+----------+-----------+-------------------+---------------+------------------------+----------------------------+-----------------------------+-----------------+--------------------+
|person_id| measurement_id|specimen_id|measurement_concept_id|measurement_date|measurement_datetime|     order_datetime|measurement_time|measurement_type_concept_id|operator_concept_id|value_as_number|value_as_string|value_as_concept_id|unit_concept_id|range_low|range_high|provider_id|visit_occurrence_id|visit_detail_id|measurement_source_value|measurement_source_value_alt|measurement_source_concept_id|unit_source_value|  value_source_value|
+---------+---------------+-----------+----------------------+----------------+--------------------+--------

In [6]:
measurement2 = measurement.where(measurement.value_source_value.contains('GCS'))
measurement2.show()

22/06/17 15:28:35 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
22/06/17 15:28:35 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
22/06/17 15:28:35 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
22/06/17 15:28:36 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
22/06/17 15:28:39 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
22/06/17 15:28:56 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB


+---------+--------------+-----------+----------------------+----------------+--------------------+--------------+----------------+---------------------------+-------------------+---------------+---------------+-------------------+---------------+---------+----------+-----------+-------------------+---------------+------------------------+----------------------------+-----------------------------+-----------------+------------------+
|person_id|measurement_id|specimen_id|measurement_concept_id|measurement_date|measurement_datetime|order_datetime|measurement_time|measurement_type_concept_id|operator_concept_id|value_as_number|value_as_string|value_as_concept_id|unit_concept_id|range_low|range_high|provider_id|visit_occurrence_id|visit_detail_id|measurement_source_value|measurement_source_value_alt|measurement_source_concept_id|unit_source_value|value_source_value|
+---------+--------------+-----------+----------------------+----------------+--------------------+--------------+----------

In [17]:
mrn_map = merge_files("/home/jupyter/omop-ed-datapipeline/mrn_map")

(696214, 2)
+---------+---------+
|person_id|      mrn|
+---------+---------+
|   484719|MR1511173|
|   496009|MR4073058|
|  1040349|MR1589839|
|  3123054|MR3942177|
|  3394543|MR3953845|
|  3476720|MR4008318|
|  3480401|MR4319480|
|  3734435| MR908529|
|  3945041| MR872122|
|  4017563|MR5277328|
|  4017563|MR1698168|
|  4704414| MR711126|
|  4751919|MR5085463|
|  4751919|MR2262124|
|  4871694|MR2861079|
|  4938663|MR3941488|
|  4965162|MR4320907|
|  4965162|MR6907366|
|  5001004|MR5566294|
|  5001004|MR2851325|
+---------+---------+
only showing top 20 rows



In [18]:
note = merge_files("/home/jupyter/omop-ed-datapipeline/note")

(7801597, 16)


22/06/07 18:15:17 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
22/06/07 18:15:17 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


+---------+----------+-------------+---------------------------+----------+-------------------+--------------------+---------------------+-------------------+--------------------+-------------------+-------------------+-----------+-------------------+---------------+-----------------+
|person_id|   note_id|note_event_id|note_event_field_concept_id| note_date|      note_datetime|note_type_concept_id|note_class_concept_id|         note_title|           note_text|encoding_concept_id|language_concept_id|provider_id|visit_occurrence_id|visit_detail_id|note_source_value|
+---------+----------+-------------+---------------------------+----------+-------------------+--------------------+---------------------+-------------------+--------------------+-------------------+-------------------+-----------+-------------------+---------------+-----------------+
| 13479732|  32302211|         null|                       null|2013-02-03|2013-02-03 18:01:00|                null|                 null|  ED

In [19]:
observation = merge_files("/home/jupyter/omop-ed-datapipeline/observation")

(216584154, 23)


22/06/07 18:16:03 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
22/06/07 18:16:03 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


+---------+--------------+-----------+----------------------+----------------+--------------------+---------------------------+---------------+---------------+-------------------+--------------------+---------------+-----------+-------------------+---------------+------------------------+-----------------------------+-----------------+----------------------+--------------------+--------------------------+-----------------+--------------------+
|person_id|observation_id|specimen_id|observation_concept_id|observation_date|observation_datetime|observation_type_concept_id|value_as_number|value_as_string|value_as_concept_id|qualifier_concept_id|unit_concept_id|provider_id|visit_occurrence_id|visit_detail_id|observation_source_value|observation_source_concept_id|unit_source_value|qualifier_source_value|observation_event_id|obs_event_field_concept_id|value_as_datetime|  value_source_value|
+---------+--------------+-----------+----------------------+----------------+--------------------+-----

In [20]:
payer_plan_period = merge_files("/home/jupyter/omop-ed-datapipeline/payer_plan_period")

(5733534, 23)
+---------+--------------------+------------------+----------------------------+--------------------------+----------------+---------------+-------------------+------------------+----------------------+--------------------+-----------------------+------------------+----------------------+---------------------+--------------------------+--------------------+-------------------------+-------------------+------------------------+-----------------------------+--------------------+------------------+
|person_id|payer_plan_period_id|contract_person_id|payer_plan_period_start_date|payer_plan_period_end_date|payer_concept_id|plan_concept_id|contract_concept_id|sponsor_concept_id|stop_reason_concept_id|  payer_source_value|payer_source_concept_id| plan_source_value|plan_source_concept_id|contract_source_value|contract_source_concept_id|sponsor_source_value|sponsor_source_concept_id|family_source_value|stop_reason_source_value|stop_reason_source_concept_id|phenotype_start_date|phen

22/06/07 18:16:11 WARN DAGScheduler: Broadcasting large task binary with size 1081.9 KiB


In [21]:
person = merge_files("/home/jupyter/omop-ed-datapipeline/person")

(456302, 20)
+---------+-----------------+-------------+--------------+------------+-------------+-------------------+--------------+---------------+--------------------+--------------------+-----------+------------+-------------------+-------------------+------------------------+--------------------+----------------------+----------------------+---------------------------+
|person_id|gender_concept_id|year_of_birth|month_of_birth|day_of_birth|date_of_birth|     birth_datetime|death_datetime|race_concept_id|ethnicity_concept_id|         location_id|provider_id|care_site_id|person_source_value|gender_source_value|gender_source_concept_id|   race_source_value|race_source_concept_id|ethnicity_source_value|ethnicity_source_concept_id|
+---------+-----------------+-------------+--------------+------------+-------------+-------------------+--------------+---------------+--------------------+--------------------+-----------+------------+-------------------+-------------------+----------------

In [22]:
phenotype = merge_files("/home/jupyter/omop-ed-datapipeline/phenotype")

(1471121, 10)
+-------------+---------+----+-------------------+---------+--------+-----------+-------------------+--------------------+------------------+
|phenotype_uri|person_id| mrn|visit_occurrence_id|      csn|order_id|specimen_id|     birth_datetime|phenotype_start_date|phenotype_end_date|
+-------------+---------+----+-------------------+---------+--------+-----------+-------------------+--------------------+------------------+
|YNHH ED Visit|   145190|null|           67523094|148142170|    null|       null|1957-04-20 05:00:00|          2017-03-24|        2017-03-24|
|YNHH ED Visit|   205775|null|           71086526|151722211|    null|       null|1956-12-26 05:00:00|          2017-05-29|        2017-05-29|
|YNHH ED Visit|   776885|null|           73009461|153671482|    null|       null|1967-05-12 04:00:00|          2017-07-04|        2017-07-04|
|YNHH ED Visit|  2814640|null|           16421582| 95326372|    null|       null|2013-10-26 04:00:00|          2014-01-10|        2014

In [23]:
procedure_occurrence = merge_files("/home/jupyter/omop-ed-datapipeline/procedure_occurrence")

(20060410, 16)
+---------+-----------------------+--------------------+-------------------+--------------+-------------------+-------------------------+-------------------+--------+-----------+-------------------+---------------+----------------------+---------------------------+---------------------+-------------------+
|person_id|procedure_occurrence_id|procedure_concept_id|procedure_source_id|procedure_date| procedure_datetime|procedure_type_concept_id|modifier_concept_id|quantity|provider_id|visit_occurrence_id|visit_detail_id|procedure_source_value|procedure_source_concept_id|modifier_source_value|status_source_value|
+---------+-----------------------+--------------------+-------------------+--------------+-------------------+-------------------------+-------------------+--------+-----------+-------------------+---------------+----------------------+---------------------------+---------------------+-------------------+
|  5504958|             e524388602|             2514436|     

In [24]:
relationship = merge_files("/home/jupyter/omop-ed-datapipeline/relationship")

(656, 6)
+--------------------+--------------------+---------------+----------------+-----------------------+-----------------------+
|     relationship_id|   relationship_name|is_hierarchical|defines_ancestry|reverse_relationship_id|relationship_concept_id|
+--------------------+--------------------+---------------+----------------+-----------------------+-----------------------+
|   Has specimen proc|Has specimen proc...|              0|               0|       Specimen proc of|               44818775|
| Has specimen source|Has specimen sour...|              0|               0|   Specimen identity of|               44818776|
|  Has specimen morph|Has specimen sour...|              0|               0|      Specimen morph of|               44818777|
|   Has specimen topo|Has specimen sour...|              0|               0|       Specimen topo of|               44818778|
|  Has specimen subst|Has specimen subs...|              0|               0|      Specimen subst of|               4

In [25]:
specimen = merge_files("/home/jupyter/omop-ed-datapipeline/specimen")

(3317779, 17)
+---------+-----------+-------------------+------------------------+-------------+-------------------+--------+---------------+------------------------+-------------------------+-------------------------------+------------------+---------------------+-----------------+--------------------------+---------------------------+---------------------------------+
|person_id|specimen_id|specimen_concept_id|specimen_type_concept_id|specimen_date|  specimen_datetime|quantity|unit_concept_id|anatomic_site_concept_id|disease_status_concept_id|collection_container_concept_id|specimen_source_id|specimen_source_value|unit_source_value|anatomic_site_source_value|disease_status_source_value|collection_container_source_value|
+---------+-----------+-------------------+------------------------+-------------+-------------------+--------+---------------+------------------------+-------------------------+-------------------------------+------------------+---------------------+-----------------

In [27]:
# Stage has no files
# stage = merge_files("/home/jupyter/omop-ed-datapipeline/stage")

In [28]:
visit_occurrence = merge_files("/home/jupyter/omop-ed-datapipeline/visit_occurrence")

(5161358, 17)
+---------+-------------------+----------------+----------------+--------------------+--------------+-------------------+---------------------+-----------+------------+--------------------+-----------------------+------------------------+--------------------------+-------------------------+-----------------------+-----------------------------+
|person_id|visit_occurrence_id|visit_concept_id|visit_start_date|visit_start_datetime|visit_end_date| visit_end_datetime|visit_type_concept_id|provider_id|care_site_id|  visit_source_value|visit_source_concept_id|admitted_from_concept_id|admitted_from_source_value|discharge_to_source_value|discharge_to_concept_id|preceding_visit_occurrence_id|
+---------+-------------------+----------------+----------------+--------------------+--------------+-------------------+---------------------+-----------+------------+--------------------+-----------------------+------------------------+--------------------------+-------------------------+---

In [29]:
vocabulary = merge_files("/home/jupyter/omop-ed-datapipeline/vocabulary")

(101, 5)
+--------------------+--------------------+--------------------+--------------------+---------------------+
|       vocabulary_id|     vocabulary_name|vocabulary_reference|  vocabulary_version|vocabulary_concept_id|
+--------------------+--------------------+--------------------+--------------------+---------------------+
|                None|OMOP Standardized...|      OMOP generated|      v5.0 04-FEB-22|             44819096|
|          Visit Type|     OMOP Visit Type|      OMOP generated|                null|             44819150|
|                 OSM|       OpenStreetMap|https://www.opens...|OSM Release 2019-...|                32541|
|           Note Type|      OMOP Note Type|      OMOP generated|                null|             44819146|
|                NUCC|National Uniform ...|http://www.nucc.o...|     2018-06-26 NUCC|             44819137|
|             Episode|        OMOP Episode|      OMOP generated|    Episode 20201014|                32523|
|                CP

In [33]:
all_frames = [care_site, concept, concept_ancestor, concept_class, concept_relationship, concept_synonym, condition_occurrence, csn_map, domain, drug_exposure, drug_strength, location, measurement, mrn_map, note, observation, payer_plan_period, person, phenotype, procedure_occurrence, relationship, specimen, visit_occurrence, vocabulary]
all_names = ['care_site', 'concept', 'concept_ancestor', 'concept_class', 'oncept_relationship', 'concept_synonym', 'condition_occurrence', 'csn_map', 'domain', 'drug_exposure', 'drug_strength', 'location', 'measurement', 'mrn_map', 'note', 'observation', 'payer_plan_period', 'person', 'phenotype', 'procedure_occurrence', 'relationship', 'specimen', 'visit_occurrence', 'vocabulary']

In [46]:
frame_to_columns = {}
for x in range(len(all_frames)):
    frame = all_frames[x]
    name = all_names[x]
    # print(name)
    cols = []
    for col in frame.dtypes:
        cols.append(col[0])
        # print(col[0]+" , "+col[1])
    # print(cols)
    frame_to_columns[name] = cols
frame_to_columns

{'care_site': ['care_site_id',
  'care_site_name',
  'place_of_service_concept_id',
  'location_id',
  'care_site_source_value',
  'place_of_service_source_value'],
 'concept': ['concept_id',
  'concept_name',
  'domain_id',
  'vocabulary_id',
  'concept_class_id',
  'standard_concept',
  'concept_code',
  'valid_start_date',
  'valid_end_date',
  'invalid_reason'],
 'concept_ancestor': ['ancestor_concept_id',
  'descendant_concept_id',
  'min_levels_of_separation',
  'max_levels_of_separation'],
 'concept_class': ['concept_class_id',
  'concept_class_name',
  'concept_class_concept_id'],
 'oncept_relationship': ['concept_id_1',
  'concept_id_2',
  'relationship_id',
  'valid_start_date',
  'valid_end_date',
  'invalid_reason'],
 'concept_synonym': ['concept_id',
  'concept_synonym_name',
  'language_concept_id'],
 'condition_occurrence': ['person_id',
  'condition_occurrence_id',
  'condition_occurrence_source_id',
  'condition_concept_id',
  'condition_start_date',
  'condition_start